In [1]:
import os
import lib

import numpy as np
import pandas as pd 

from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:
SEED = 42
VL_SIZE = 0.2

In [4]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

learn_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.RandomResizedCrop(224),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ColorJitter(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

test_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])


learn_data = datasets.ImageFolder('./data/train', transform=learn_transform)
splits = [ int((1-VL_SIZE)*len(learn_data)), int((VL_SIZE)*len(learn_data)) ]

tr_data, vl_data = random_split(learn_data, splits, generator=torch.Generator().manual_seed(SEED))

tr_loader = DataLoader(tr_data, batch_size=64, shuffle=True)
vl_loader = DataLoader(vl_data, batch_size=64, shuffle=True)

In [5]:
model = models.densenet161(pretrained=True)
model.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [6]:
model = lib.freeze_parameters(model)

In [7]:
classifier = nn.Sequential(
  nn.Linear(in_features=2208, out_features=2208),
  nn.ReLU(),
  nn.Dropout(p=0.4),
  nn.Linear(in_features=2208, out_features=1024),
  nn.ReLU(),
  nn.Dropout(p=0.3),
  nn.Linear(in_features=1024, out_features=8),
  nn.LogSoftmax(dim=1)  
)
    
model.classifier = classifier
model.classifier

Sequential(
  (0): Linear(in_features=2208, out_features=2208, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=2208, out_features=1024, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=1024, out_features=8, bias=True)
  (7): LogSoftmax(dim=1)
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [9]:
epoch = 5+5

In [11]:
model, train_loss, test_loss = lib.train(model, tr_loader, vl_loader, epoch, optimizer, criterion)

Epoch: 1/10


/Users/a1/Work/ml_comps/venv/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 